In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.master("local").appName("Test Spark").getOrCreate()

sc=spark.sparkContext

In [3]:
spark

In [4]:


spark = SparkSession.builder.appName('movie_recommendation').getOrCreate()

name_basics_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('name.basics.tsv.gz')
title_akas_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('title.akas.tsv.gz')
title_basics_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('title.basics.tsv.gz')
title_crew_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('title.crew.tsv.gz')
title_episode_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('title.episode.tsv.gz')
title_principals_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('title.principals.tsv.gz')
title_ratings_df = spark.read.option("header",True).option("sep","\t").option("inferSchema",True).csv('title.ratings.tsv.gz')

# Check the contents of the DataFrames:
name_basics_df.show()
title_akas_df.show()
title_basics_df.show()
title_crew_df.show()
title_episode_df.show()
title_principals_df.show()
title_ratings_df.show()


+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0053137,tt00504...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0075213,tt00383...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0056404,tt00544...|
|nm0000004|       John Belushi|     1949|     1982|actor,soundtrack,...|tt0072562,tt00787...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0060827,tt00694...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0038787,tt00345...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0043265,tt00345...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [5]:
# Drop the 'deathYear' column
name_basics_df = name_basics_df.drop('deathYear')

# Remove rows with missing values
name_basics_df = name_basics_df.na.drop()

# Drop irrelevant columns
title_akas_df = title_akas_df.drop('ordering', 'attributes')

# Remove rows with missing values
title_akas_df = title_akas_df.na.drop()

# Drop irrelevant columns
title_basics_df = title_basics_df.drop('isAdult', 'endYear')

# Remove rows with missing values
title_basics_df = title_basics_df.na.drop()

# Drop irrelevant columns
title_crew_df = title_crew_df.drop('writers')

# Remove rows with missing values
title_crew_df = title_crew_df.na.drop()

# Remove rows with missing values
title_episode_df = title_episode_df.na.drop()

# Remove rows with missing values
title_principals_df = title_principals_df.na.drop()

# Remove rows with missing values
title_ratings_df = title_ratings_df.na.drop()

# Check the contents of the DataFrames:
name_basics_df.show()
title_akas_df.show()
title_basics_df.show()
title_crew_df.show()
title_episode_df.show()
title_principals_df.show()
title_ratings_df.show()

+---------+-------------------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|soundtrack,actor,...|tt0053137,tt00504...|
|nm0000002|      Lauren Bacall|     1924|  actress,soundtrack|tt0075213,tt00383...|
|nm0000003|    Brigitte Bardot|     1934|actress,soundtrac...|tt0056404,tt00544...|
|nm0000004|       John Belushi|     1949|actor,soundtrack,...|tt0072562,tt00787...|
|nm0000005|     Ingmar Bergman|     1918|writer,director,a...|tt0060827,tt00694...|
|nm0000006|     Ingrid Bergman|     1915|actress,soundtrac...|tt0038787,tt00345...|
|nm0000007|    Humphrey Bogart|     1899|actor,soundtrack,...|tt0043265,tt00345...|
|nm0000008|      Marlon Brando|     1924|actor,soundtrack,...|tt0047296,tt00787...|
|nm0000009|     Richard Burton|     1925|actor,soundtrack,...|tt0087803,tt00

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create Spark session
spark = SparkSession.builder.getOrCreate()



# Merge relevant data frames
merged_df = title_basics_df.join(title_ratings_df, on='tconst')
merged_df = title_principals_df.join(merged_df, on='tconst')

# Filter and select necessary columns
als_data = merged_df.select(col('nconst').alias('user_id'), col('tconst').alias('item_id'), col('averageRating').alias('rating'))

# Optionally preprocess the data

# Handle missing values
als_data = als_data.na.drop()

# Normalize ratings (if needed)
als_data = als_data.withColumn('rating', (col('rating') - als_data.agg({'rating': 'mean'}).collect()[0][0]) / als_data.agg({'rating': 'stddev'}).collect()[0][0])

# Encode categorical variables (if needed)
# You might need to convert categorical variables into numerical representations using techniques like one-hot encoding or label encoding

# Save the ALS input data to a file or use it for training the ALS model
# als_data.write.csv('als_input.csv', header=True)




In [7]:
# Show the first 10 rows of the merged inputs
als_data.show(10)

+---------+---------+-------------------+
|  user_id|  item_id|             rating|
+---------+---------+-------------------+
|nm0653028|tt0000008|-1.1398643155101096|
|nm0005690|tt0000008|-1.1398643155101096|
|nm0374658|tt0000008|-1.1398643155101096|
|nm0721526|tt0000015|-0.6220574734081569|
|nm0932055|tt0000019|-1.3617815335538042|
|nm0617588|tt0000051|-2.6193124358014046|
|nm0874739|tt0000058| -2.175477999714016|
|nm0617588|tt0000058| -2.175477999714016|
|nm0617588|tt0000065|-2.8412296538450987|
|nm0617588|tt0000095|-1.7316435636266279|
+---------+---------+-------------------+
only showing top 10 rows



In [7]:
als_data.filter(col("rating").isNull()).show()

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
+-------+-------+------+



In [9]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


from pyspark.sql.functions import col

from pyspark.sql.functions import abs

from pyspark.sql.functions import isnan

# Drop rows with null values
als_data = als_data.dropna()

# Convert ratings to non-negative values
als_data = als_data.withColumn("rating", abs(als_data["rating"]))


# Filter out rows with null user_id
als_data = als_data.filter(als_data['user_id'].isNotNull())

# Filter out rows with null ratings
als_data = als_data.filter(als_data['rating'].isNotNull())

# Filter out rows with null item_id
als_data = als_data.filter(als_data['item_id'].isNotNull())

# Convert user_id and item_id columns to numeric type
als_data = als_data.withColumn("user_id", col("user_id").cast("integer"))
als_data = als_data.withColumn("item_id", col("item_id").cast("integer"))

# Create ALS model
als = ALS(
    userCol="user_id",
    itemCol="item_id",
    ratingCol="rating",
    coldStartStrategy="drop"
)

# Split the data into training and test sets
(training, test) = als_data.randomSplit([0.8, 0.2])

# Fit the ALS model
model = als.fit(training)

# Generate movie recommendations for all users
userRecs = model.recommendForAllUsers(10)  # Generate 10 recommendations for each user

# Output the recommendations for each user
userRecs.show(truncate=False)





IllegalArgumentException: requirement failed: No ratings available from MapPartitionsRDD[241] at map at ALS.scala:729

In [10]:
als_data.filter(col("rating").isNull()).show()


+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
+-------+-------+------+

